In [ ]:
import select
import psycopg2
import psycopg2.extensions
from twitter import Twitter, OAuth
import os

os.environ["https_proxy"] = os.environ["http_proxy"]
token = '4928196327-OJO2bOcQ8ODxZcvQqcQZOJ1R9WW0N2F4rrzSnhk'
token_secret = 'ZVW5ZwNXdp8zn7MxlOnLtlIctPIRYwq52L8FLGwTQVamr'
consumer_key = 'Gk8b97BvqMaYnzAG2uUpVxAO4'
consumer_secret = 'oXlYxQCgW0uYkJXHbHwuEnZ2sbMEp98hhVg8kLn732eiS1oGrG'
twitter = Twitter(auth=OAuth(token, token_secret, consumer_key, consumer_secret))

field_names =  ['user_screen_name', 'user_name', 'id_str', 'created_at', 
                'sentiment', 'categories', 'text', 'klout_score', 'segment_id',
               'got_reply', 'followers_count', 'is_agent_reply']

segment_dict = {0:'-', 1:'Mass Market', 2:'Young Professional',
                3:'Mass Afluent', 4:'Affluent', 5:'High Net Worth'}

def get_priority_key(tw_l):
    for tw in tw_l:
        if not tw['is_agent_reply']:
            p = (tw['segment_id']/2 - tw['sentiment'] - tw['got_reply']*5 +
                tw['followers_count']/1000 + int('fraud' in tw['categories']))
            return p

In [ ]:
from flask import Flask, render_template, jsonify, request
from flask_sockets import Sockets
from json import dumps
import gevent
import time
from gevent.pool import Pool


app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            ws.send("{notification:\'new tweet\'}")
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()

@app.route('/reply', methods=['POST'])
def profile():
    data = eval(request.form['py_data'])
    status = request.form['status']
    status_id = request.form['status_id']
    
    sentiment = data['sentiment']
    user_name = data['user_name']
    
    tweet = twitter.statuses.update(status=status, in_reply_to_status_id=status_id)
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    
    curs.execute("""update customers
                set got_reply=1
                where user_screen_name=%s""",(tweet['in_reply_to_screen_name'],))
    
    curs.execute("""insert into stream (id_str, text, created_at, in_reply_to_status_id,
                in_reply_to_screen_name, user_screen_name, user_name, sentiment, is_agent_reply)
                values (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                 (tweet['id_str'],
                  tweet['text'],
                  tweet['created_at'],
                  tweet['in_reply_to_status_id'],
                  tweet['in_reply_to_screen_name'],
                  tweet['in_reply_to_screen_name'],
                  user_name,
                  sentiment,
                  1,))
    
    conn.close()

    return 'good'
    
@app.route('/data', methods=['GET'])
def provide_data():
    #https://twitter.com/statuses/ID
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()

    curs.execute("""select stream.user_screen_name, stream.user_name, stream.id_str,
                stream.created_at, stream.sentiment, stream.categories, stream.text,
                customers.klout_score, customers.segment, customers.got_reply, 
                customers.followers_count, stream.is_agent_reply
                from stream
                INNER JOIN customers
                ON customers.user_screen_name=stream.user_screen_name
                where stream.user_screen_name in 
                (
                    select tmp.user_screen_name
                    from (
                        select user_screen_name as user_screen_name, max(id_str) as max_id_str
                        from stream
                        group by user_screen_name
                    ) tmp
                    left join (
                        select user_screen_name, got_reply
                        from customers
                    ) tmp2
                    on tmp.user_screen_name=tmp2.user_screen_name
                    order by tmp2.got_reply asc, tmp.max_id_str desc limit 10
                )
                order by customers.got_reply desc, stream.id_str desc
                ;""")
    rec = curs.fetchall()
    conn.close()
    
    user_blocks = {v[0]:[] for v in rec}
    for row in rec:
        if len(user_blocks[row[0]]) < 5:
            user_blocks[row[0]] += [{k:v for k,v in zip(field_names, row)}]
            user_blocks[row[0]][-1]['id_str'] = str(user_blocks[row[0]][-1]['id_str'])
            user_blocks[row[0]][-1]['segment'] = segment_dict[user_blocks[row[0]][-1]['segment_id']]
    
    user_blocks_v = user_blocks.values()
    user_blocks_v_s = sorted(user_blocks_v, key=get_priority_key)
    
    return jsonify({'data':user_blocks_v_s})

@app.route('/')
def homepage():
    try:
        return render_template("index.html")
    except Exception, e:
        return str(e)

if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8090), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()